# Compilation for ccLight

This notebook provides an introduction to compilation for ccLight with an emphasis on:

- hybrid quantum/classical code generation
- getting measurement results
- control-flow in terms of:
    - if, if-else
    - for loop
    - do-while loop


In [1]:
def print_file(fname):
    with open(fname) as f:
        read_data = f.read()
        print(read_data)
        
    f.close()

In [2]:
from openql import openql as ql
import os

curdir = '.'
output_dir = os.path.join(curdir, 'test_output')
ql.set_option('output_dir', output_dir)
config_fn = os.path.join(curdir, '../../tests/hardware_config_cc_light.json')
platform = ql.Platform("myPlatform", config_fn)

In [3]:
# This is what we could do till last demo

def test():
    sweep_points = [1]
    nqubits = 5
    p = ql.Program("aProgram", platform, nqubits)
    p.set_sweep_points(sweep_points, len(sweep_points))
    k = ql.Kernel("aKernel", platform, nqubits)

    k.gate('x', [0])
    k.gate('h', [1])
    k.gate('cz', [2, 0])
    k.gate('measure', [0])
    k.gate('measure', [1])

    p.add_kernel(k)
    p.compile()

In [4]:
test()
print_file(os.path.join(output_dir,'aProgram.qisa'))

smis s0, {0} 
smis s1, {1} 
smis s2, {2} 
smis s3, {3} 
smis s4, {4} 
smis s5, {5} 
smis s6, {6} 
smis s7, {0, 1, 2, 3, 4, 5, 6} 
smis s8, {0, 1, 5, 6} 
smis s9, {2, 3, 4} 
smit t0, {(2, 0)} 
start:

aKernel:
    1    x s0
    2    cz t0 | h s1
    2    measz s1
    2    measz s0
    qwait 2

    br always, start
    nop 
    nop




In [5]:
def test_classical():
    sweep_points = [1]
    num_qubits = 5
    num_cregs = 10
    p = ql.Program("classical_program", platform, num_qubits, num_cregs)
    p.set_sweep_points(sweep_points, len(sweep_points))
    k = ql.Kernel("aKernel", platform, num_qubits, num_cregs)

    # create classical registers
    rd = ql.CReg()
    rs1 = ql.CReg()
    rs2 = ql.CReg()

    # add quantum operations
    k.gate('h', [0])
    k.gate('h', [1])
    k.gate('measure', [0], rs1)
    k.gate('measure', [1], rs2)
    
    # add classical operations
    k.classical(rd, ql.Operation(rs1, '==', rs2))

    p.add_kernel(k)
    p.compile()

In [6]:
test_classical()
print_file(os.path.join(output_dir,'classical_program.qisa'))

smis s0, {0} 
smis s1, {1} 
smis s2, {2} 
smis s3, {3} 
smis s4, {4} 
smis s5, {5} 
smis s6, {6} 
smis s7, {0, 1, 2, 3, 4, 5, 6} 
smis s8, {0, 1, 5, 6} 
smis s9, {2, 3, 4} 
smis s10, {0, 1} 
start:

aKernel:
    1    h s10
    2    measz s0
    qwait 1
    qwait 1
    fmr r1, q0
    1    measz s1
    qwait 1
    qwait 1
    fmr r2, q1
    cmp r1, r2
    nop
    fbr EQ, r0

    br always, start
    nop 
    nop




In [7]:
def test_for():
    num_qubits = 5

    sweep_points = [1,2]
    p = ql.Program('test_for', platform, num_qubits)
    p.set_sweep_points(sweep_points, len(sweep_points))

    k1 = ql.Kernel('aKernel1', platform, num_qubits)

    k1.gate('x', [0])
    k1.gate('y', [0])

    p.add_for(k1, 10)
    p.compile()

In [8]:
test_for()
print_file(os.path.join(output_dir,'test_for.qisa'))

smis s0, {0} 
smis s1, {1} 
smis s2, {2} 
smis s3, {3} 
smis s4, {4} 
smis s5, {5} 
smis s6, {6} 
smis s7, {0, 1, 2, 3, 4, 5, 6} 
smis s8, {0, 1, 5, 6} 
smis s9, {2, 3, 4} 
start:

aKernel1_for0_start:
    ldi r29, 10
    ldi r30, 1
    ldi r31, 0

aKernel1:
    1    x s0
    2    y s0
    qwait 2

aKernel1_for0_end:
    add r31, r31, r30
    cmp r31, r29
    nop
    br lt, aKernel1

    br always, start
    nop 
    nop




In [9]:
def test_if_else():
    num_qubits = 5
    num_cregs = 10

    sweep_points = [1,2]
    p = ql.Program('test_if_else', platform, num_qubits, num_cregs)
    p.set_sweep_points(sweep_points, len(sweep_points))

    k1 = ql.Kernel('aKernel1', platform, num_qubits, num_cregs)
    k2 = ql.Kernel('aKernel2', platform, num_qubits, num_cregs)

    # create classical registers
    rs1 = ql.CReg()
    rs2 = ql.CReg()

    # quanutm operations
    k1.gate('x', [0])
    k2.gate('y', [0])

    # simple if
    p.add_if_else(k1, k2, ql.Operation(rs1, '==', rs2))

    p.compile()

In [10]:
test_if_else()
print_file(os.path.join(output_dir,'test_if_else.qisa'))

smis s0, {0} 
smis s1, {1} 
smis s2, {2} 
smis s3, {3} 
smis s4, {4} 
smis s5, {5} 
smis s6, {6} 
smis s7, {0, 1, 2, 3, 4, 5, 6} 
smis s8, {0, 1, 5, 6} 
smis s9, {2, 3, 4} 
start:

aKernel1_if1:
    cmp r2, r1
    nop
    br ne, aKernel1_if1_end

aKernel1:
    1    x s0
    qwait 2

aKernel1_if1_end:

aKernel2_else1:
    cmp r2, r1
    nop
    br eq, aKernel2_else1_end

aKernel2:
    1    y s0
    qwait 2

aKernel2_else1_end:

    br always, start
    nop 
    nop




In [11]:
def test_do_while():
    num_qubits = 5
    num_cregs = 10

    p = ql.Program('test_do_while', platform, num_qubits, num_cregs)
    sweep_points = [1,2]
    p.set_sweep_points(sweep_points, len(sweep_points))

    k = ql.Kernel('aKernel1', platform, num_qubits, num_cregs)

    # create classical registers
    rs1 = ql.CReg()
    rs2 = ql.CReg()

    # quanutm operations
    k.gate('h', [0])
    k.gate('h', [1])
    k.gate('measure', [0], rs1)
    k.gate('measure', [1], rs2)
    
    p.add_do_while(k, ql.Operation(rs1, '==', rs2))

    p.compile()

In [12]:
test_do_while()
print_file(os.path.join(output_dir,'test_do_while.qisa'))

smis s0, {0} 
smis s1, {1} 
smis s2, {2} 
smis s3, {3} 
smis s4, {4} 
smis s5, {5} 
smis s6, {6} 
smis s7, {0, 1, 2, 3, 4, 5, 6} 
smis s8, {0, 1, 5, 6} 
smis s9, {2, 3, 4} 
smis s10, {0, 1} 
start:

aKernel1_do_while2_start:

aKernel1:
    1    h s10
    2    measz s0
    qwait 1
    qwait 1
    fmr r1, q0
    1    measz s1
    qwait 1
    qwait 1
    fmr r2, q1

aKernel1_do_while2:
    cmp r1, r2
    nop
    br eq, aKernel1_do_while2_start

    br always, start
    nop 
    nop




In [13]:
def test_nested():
    num_qubits = 5
    num_cregs = 10

    p = ql.Program('test_nested', platform, num_qubits, num_cregs)
    sweep_points = [1,2]
    p.set_sweep_points(sweep_points, len(sweep_points))

    sp1 = ql.Program('whileProg', platform, num_qubits, num_cregs)
    sp2 = ql.Program('forProg', platform, num_qubits, num_cregs)

    k1 = ql.Kernel('aKernel1', platform, num_qubits, num_cregs)

    # create classical registers
    rs1 = ql.CReg()
    rs2 = ql.CReg()

    # quanutm operations
    k1.gate('x', [0])
    k1.gate('y', [1])
    k1.gate('measure', [0], rs1)
    k1.gate('measure', [1], rs2)

    sp1.add_do_while(k1, ql.Operation(rs1, '>=', rs2))
    sp2.add_for(sp1, 100)
    p.add_program(sp2)

    p.compile()

In [15]:
test_nested()
print_file(os.path.join(output_dir,'test_nested.qisa'))

smis s0, {0} 
smis s1, {1} 
smis s2, {2} 
smis s3, {3} 
smis s4, {4} 
smis s5, {5} 
smis s6, {6} 
smis s7, {0, 1, 2, 3, 4, 5, 6} 
smis s8, {0, 1, 5, 6} 
smis s9, {2, 3, 4} 
start:

whileProg_for6_start:
    ldi r29, 100
    ldi r30, 1
    ldi r31, 0

whileProg:

aKernel1_do_while5_start:

aKernel1:
    1    x s0
    2    measz s0 | y s1
    qwait 1
    qwait 1
    fmr r1, q0
    1    measz s1
    qwait 1
    qwait 1
    fmr r2, q1

aKernel1_do_while5:
    cmp r1, r2
    nop
    br ge, aKernel1_do_while5_start

whileProg_for6_end:
    add r31, r31, r30
    cmp r31, r29
    nop
    br lt, whileProg

    br always, start
    nop 
    nop


